In [ ]:
!pip install -Uqq fastbook
!pip list -v | grep fastai

     |████████████████████████████████| 727kB 8.4MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 1.1MB 15.2MB/s 
     |████████████████████████████████| 194kB 30.1MB/s 
     |████████████████████████████████| 61kB 10.2MB/s 


In [ ]:
from PIL import ImageFile

import pandas as pd
import re

from pathlib import Path
from datetime import datetime

from fastbook import *
from fastai import * 
from fastai.vision.widgets import *
from fastai.callback.fp16 import *

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Raw data

In [ ]:
path = Path('/content/drive/MyDrive/MACHINLEARNING/DesafioAgTech2020/data/train_heatmap_ts')
test = Path('/content/drive/MyDrive/MACHINLEARNING/DesafioAgTech2020/data/test_heatmap_ts')

In [ ]:
path.exists()

True

In [ ]:
etiquetas_url = 'https://raw.githubusercontent.com/DesafiosAgTech/DesafioAgTech2020/master/dataset/Etiquetas.csv'
etiquetas =  pd.read_csv(etiquetas_url, error_bad_lines=False)

## Parameters

Parameters de arquitectura y de training

In [ ]:
model_id = "xresnet50_heatmap"

In [ ]:
# instantiate balanced accuracy metric
balanced_accuracy = BalancedAccuracy()

# dataloader
seed = 1 # seed split train-validation
valid_pct = 0.2 # % data validacion
bs = 32 # batch size

# fine_tune model parameters
arch =  xresnet50  # architecture
loss_func = CrossEntropyLossFlat
pretrained = True
metrics = [balanced_accuracy, error_rate]
epochs = 22
lr = 10e-3
# opt_func = Adam (valor default)

## Data

In [ ]:
# Load data with fastai
lotes2 = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=valid_pct, seed=seed),
    get_y=parent_label,
    item_tfms=RatioResize(224))
# square of resize pixels. Bigger the better but more hardware needed pg28 of fastai

dls = lotes2.dataloaders(path, bs=bs)

Controles

In [ ]:
# N of training data
dls.train.n

849

In [ ]:
dls.train.vocab

['A', 'B', 'G', 'M', 'N', 'P', 'R', 'S', 'T', 'U', 'X', 'aa', 'm', 's']

In [ ]:
train_lbls = L(map(lambda x: classes[x[1]], dls.train_ds))
label_counter = Counter(train_lbls)

In [ ]:
label_counter

Counter({'A': 1,
         'B': 5,
         'G': 1,
         'M': 174,
         'N': 63,
         'P': 47,
         'R': 4,
         'S': 263,
         'T': 2,
         'U': 9,
         'X': 29,
         'aa': 1,
         'm': 4,
         's': 77})

In [ ]:
dls.valid.show_batch(max_n=1, nrows=1)

In [ ]:
dls.show_batch()

## Sample weights

In [ ]:
def make_weights_for_balanced_classes(labels, nclasses):
  """
  Returns weight of each image 
  Weights are inversely correlated to class frequency 
  """
  count = [0] * nclasses                                                      
  for l in labels:                                                         
      count[l] += 1                                                     
  weight_per_class = [0.] * nclasses                                      
  N = float(sum(count))                                                   
  for i in range(nclasses):                                                   
      weight_per_class[i] = N/float(count[i])                                 
  weight = [0] * len(labels)                                              
  for idx, val in enumerate(labels):                                          
      weight[idx] = weight_per_class[val]                                  
  return weight


def get_sampler(dataloader):
  """
  Returns sampler using weights for a given data
  Esta funcion es una alternativa a get_weights()
  """
  imagenes = list()
  labels = list()
  for i,l in dataloader.train:
    imagenes.append(i)
    labels.append(l)
  imagenes2 = torch.cat(imagenes)
  labels2 = torch.cat(labels)
  n_classes = len(torch.unique(labels2))
  weights = make_weights_for_balanced_classes(labels2, n_classes)                                                                
  weights = torch.DoubleTensor(weights)                                       
  sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))
  return sampler


def get_weights(dls):
  """
  Get weights based on the class distribution in the training data
  Basado en https://forums.fast.ai/t/oversampling-in-fastai2/73721/3
  Esta funcion es una alternativa a get_sampler()
  """
  # 0th index would provide the vocab from text
  # 1st index would provide the vocab from classes
  classes = dls.train.vocab
  #Get label ids from the dataset using map
  #train_lb_ids = L(map(lambda x: x[1], dls.train_ds))
  # Get the actual labels from the label_ids & the vocab
  #train_lbls = L(map(lambda x: classes[x], train_lb_ids))
  #Combine the above into a single
  train_lbls = L(map(lambda x: classes[x[1]], dls.train_ds))
  label_counter = Counter(train_lbls)
  n_most_common_class = max(label_counter.values()); 
  print(f'Occurrences of the most common class {n_most_common_class}')
  # Source: https://discuss.pytorch.org/t/what-is-the-weight-values-mean-in-torch-nn-crossentropyloss/11455/9
  # weights = [n_most_common_class/v for k, v in label_counter.items() if v > 0]; return weights 
  weights = [n_most_common_class/label_counter[k] for  k in dls.train.vocab ]
  return weights


In [ ]:
# create dataloader using sampler
dls2 =  lotes2.dataloaders(path, bs=bs, sampler=get_sampler(dls))

## Learning rate

Find "optimal" learning rate

In [ ]:
learn = None
learn = cnn_learner(
    dls2, arch, loss_func=loss_func(), pretrained=pretrained
    , metrics=metrics)
lr_min, lr_steep = learn.lr_find()
# # # 10e-3

## Train - validate

In [ ]:
# def set_seeds():
#   random.seed(42)
#   np.random.seed(12345)
#   torch.manual_seed(1234)
#   torch.backends.cudnn.deterministic = True
#   torch.backends.cudnn.benchmark = False

# set_seeds()

learn = None
learn = cnn_learner(
    dls2, arch, loss_func=loss_func(), pretrained=pretrained, metrics= metrics).to_fp16()
learn.fine_tune(epochs=epochs, base_lr=lr)
learn.recorder.plot_loss()

### Performance

Validation metrics

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
interp.most_confused(min_val=5)

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
# # loss of each validation image
# interp.losses.abs()

## Train

Final training with all data (train + validation)

In [ ]:
lotes_alldata = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.0, seed=seed),
    get_y=parent_label,
    item_tfms=RatioResize(224))

dls_alldata = lotes_alldata.dataloaders(path, bs=bs)
dls_alldata2 = lotes_alldata.dataloaders(
    path, bs=bs, sampler=get_sampler(dls_alldata))

learn = None
learn = cnn_learner(
    dls_alldata2, arch, loss_func=loss_func(), pretrained=pretrained, metrics=metrics).to_fp16()
learn.fine_tune(epochs=epochs, base_lr=lr)
learn.recorder.plot_loss()

## Save

In [ ]:
model_file = f'/content/drive/MyDrive/MACHINLEARNING/DesafioAgTech2020/{model_id}.pkl'

# save models as pkl
learn.export(model_file)

In [ ]:
def generate_submit_from_learner(
    pkt_path_str
    ,test_path_str='/content/drive/MyDrive/MACHINLEARNING/DesafioAgTech2020/data/test'
    ,export_path_str = '/content/drive/MyDrive/MACHINLEARNING/DesafioAgTech2020/submit/'):
    """
    Submits csv ready to upload in the export path. Date at the end
    Input: 
      - pkl learner path as string
      - path of test directory (.png) as string. Must end in GlobalId.png
      - path to export directory as string
    """
    etiquetas_url = 'https://raw.githubusercontent.com/DesafiosAgTech/DesafioAgTech2020/master/dataset/Etiquetas.csv'
    etiquetas =  pd.read_csv(etiquetas_url, error_bad_lines=False)
    # Load model and predict using same transformation?
    learn = load_learner(str(pkt_path_str))
    test_dl = learn.dls.test_dl(get_image_files(test_path_str))
    # Predicts and returns most probable class
    preds,y = learn.get_preds(dl = test_dl)
    y = torch.argmax(preds, dim = 1)
    # Predicted to corresponding CultivoId + add global id for submit
    glob_list = [re.findall(r'(\d+).png', r) for r in [str(p) for p in test_dl.items]]
    globalids = [int(id) for sub in glob_list for id in sub] 
    prediction = [learn.dls.vocab[p] for p in y.tolist()]
    prediction_df = pd.DataFrame(zip(globalids,prediction), columns=['globalid','clase'])
    submit = prediction_df.merge(
        etiquetas, how = "left", left_on= 'clase', right_on='Cultivo')[['globalid','CultivoId']]
    # exports submit
    now = datetime.now()
    file_name = \
      str(export_path_str) + now.strftime('%Y%m%d%H%M') + str(test_path_str).split("/")[-1] + '.csv'
    submit.to_csv(file_name, header=False, index=False)
    

In [ ]:
# submit csv of results
generate_submit_from_learner(model_file, test_path_str=str(test))